In [1]:
import rospy
from std_msgs.msg import String
from sensor_msgs.msg import Image, CameraInfo, PointCloud2
from cv_bridge import CvBridge, CvBridgeError
import message_filters
import numpy as np
from python_msg_conversions import pointclouds
import cv2

In [2]:
def depth_color(val, min_d=0, max_d=120):
    """ 
    print Color(HSV's H value) corresponding to distance(m) 
    close distance = red , far distance = blue
    """
    np.clip(val, 0, max_d, out=val) # max distance is 120m but usually not usual
    return (((val - min_d) / (max_d - min_d)) * 120).astype(np.uint8) 

def in_h_range_points(points, m, n, fov):
    """ extract horizontal in-range points """
    return np.logical_and(np.arctan2(n,m) > (-fov[1] * np.pi / 180), \
                          np.arctan2(n,m) < (-fov[0] * np.pi / 180))

def in_v_range_points(points, m, n, fov):
    """ extract vertical in-range points """
    return np.logical_and(np.arctan2(n,m) < (fov[1] * np.pi / 180), \
                          np.arctan2(n,m) > (fov[0] * np.pi / 180))

def fov_setting(points, x, y, z, dist, h_fov, v_fov):
    """ filter points based on h,v FOV  """
    
    if h_fov[1] == 180 and h_fov[0] == -180 and v_fov[1] == 2.0 and v_fov[0] == -24.9:
        return points
    
    if h_fov[1] == 180 and h_fov[0] == -180:
        return points[in_v_range_points(points, dist, z, v_fov)]
    elif v_fov[1] == 2.0 and v_fov[0] == -24.9:        
        return points[in_h_range_points(points, x, y, h_fov)]
    else:
        h_points = in_h_range_points(points, x, y, h_fov)
        v_points = in_v_range_points(points, dist, z, v_fov)
        return points[np.logical_and(h_points, v_points)]

def in_range_points(points, size):
    """ extract in-range points """
    return np.logical_and(points > 0, points < size)    

def velo_points_filter(points, v_fov, h_fov):
    """ extract points corresponding to FOV setting """
    
    # Projecting to 2D
    x = points[:, 0]
    y = points[:, 1]
    z = points[:, 2]
    dist = np.sqrt(x ** 2 + y ** 2 + z ** 2)

    if h_fov[0] < -90:
        h_fov = (-90,) + h_fov[1:]
    if h_fov[1] > 90:
        h_fov = h_fov[:1] + (90,)
    
    x_lim = fov_setting(x, x, y, z, dist, h_fov, v_fov)[:,None]
    y_lim = fov_setting(y, x, y, z, dist, h_fov, v_fov)[:,None]
    z_lim = fov_setting(z, x, y, z, dist, h_fov, v_fov)[:,None]

    # Stack arrays in sequence horizontally
    xyz_ = np.hstack((x_lim, y_lim, z_lim))
    xyz_ = xyz_.T

    # stack (1,n) arrays filled with the number 1
    one_mat = np.full((1, xyz_.shape[1]), 1)
    xyz_ = np.concatenate((xyz_, one_mat),axis = 0)

    # need dist info for points color
    dist_lim = fov_setting(dist, x, y, z, dist, h_fov, v_fov)
    color = depth_color(dist_lim, 0, 70)
    
    return xyz_, color

In [3]:
def calib_velo2cam(filepath):
    """ 
    get Rotation(R : 3x3), Translation(T : 3x1) matrix info 
    using R,T matrix, we can convert velodyne coordinates to camera coordinates
    """
    with open(filepath, "r") as f:
        file = f.readlines()    
        
        for line in file:
            (key, val) = line.split(':',1)
            if key == 'R':
                R = np.fromstring(val, sep=' ')
                R = R.reshape(3, 3)
            if key == 'T':
                T = np.fromstring(val, sep=' ')
                T = T.reshape(3, 1)
    return R, T

def calib_cam2cam(filepath, mode):
    """
    If your image is 'rectified image' :
        get only Projection(P : 3x4) matrix is enough
    but if your image is 'distorted image'(not rectified image) :
        you need undistortion step using distortion coefficients(5 : D)
        
    in this code, I'll get P matrix since I'm using rectified image
    """
    with open(filepath, "r") as f:
        file = f.readlines()
        
        for line in file:
            (key, val) = line.split(':',1)
            if key == ('P_rect_' + mode):
                P_ = np.fromstring(val, sep=' ')
                P_ = P_.reshape(3, 4)
                # erase 4th column ([0,0,0])
                P_ = P_[:3, :3]
    return P_

def velo3d_2_camera2d_points(points, v_fov, h_fov, vc_path, cc_path, mode='02'):
    """ print velodyne 3D points corresponding to camera 2D image """
    
    # R_vc = Rotation matrix ( velodyne -> camera )
    # T_vc = Translation matrix ( velodyne -> camera )
    R_vc, T_vc = vc_path
    
    # P_ = Projection matrix ( camera coordinates 3d points -> image plane 2d points )
    P_ = cc_path

    """
    xyz_v - 3D velodyne points corresponding to h, v FOV in the velodyne coordinates
    c_    - color value(HSV's Hue) corresponding to distance(m)
    
             [x_1 , x_2 , .. ]
    xyz_v =  [y_1 , y_2 , .. ]   
             [z_1 , z_2 , .. ]
             [ 1  ,  1  , .. ]
    """  
    xyz_v, c_ = velo_points_filter(points, v_fov, h_fov)
    
    """
    RT_ - rotation matrix & translation matrix
        ( velodyne coordinates -> camera coordinates )
    
            [r_11 , r_12 , r_13 , t_x ]
    RT_  =  [r_21 , r_22 , r_23 , t_y ]   
            [r_31 , r_32 , r_33 , t_z ]
    """
    RT_ = np.concatenate((R_vc, T_vc),axis = 1)
    
    # convert velodyne coordinates(X_v, Y_v, Z_v) to camera coordinates(X_c, Y_c, Z_c) 
    for i in range(xyz_v.shape[1]):
        xyz_v[:3,i] = np.matmul(RT_, xyz_v[:,i])
        
    """
    xyz_c - 3D velodyne points corresponding to h, v FOV in the camera coordinates
             [x_1 , x_2 , .. ]
    xyz_c =  [y_1 , y_2 , .. ]   
             [z_1 , z_2 , .. ]
    """ 
    xyz_c = np.delete(xyz_v, 3, axis=0)

    # convert camera coordinates(X_c, Y_c, Z_c) image(pixel) coordinates(x,y) 
    for i in range(xyz_c.shape[1]):
        xyz_c[:,i] = np.matmul(P_, xyz_c[:,i])    

    """
    xy_i - 3D velodyne points corresponding to h, v FOV in the image(pixel) coordinates before scale adjustment
    ans  - 3D velodyne points corresponding to h, v FOV in the image(pixel) coordinates
             [s_1*x_1 , s_2*x_2 , .. ]
    xy_i =   [s_1*y_1 , s_2*y_2 , .. ]        ans =   [x_1 , x_2 , .. ]  
             [  s_1   ,   s_2   , .. ]                [y_1 , y_2 , .. ]
    """
    xy_i = xyz_c[::]/xyz_c[::][2]
    ans = np.delete(xy_i, 2, axis=0)
    
    """
    width = 1242
    height = 375
    w_range = in_range_points(ans[0], width)
    h_range = in_range_points(ans[1], height)

    ans_x = ans[0][np.logical_and(w_range,h_range)][:,None].T
    ans_y = ans[1][np.logical_and(w_range,h_range)][:,None].T
    c_ = c_[np.logical_and(w_range,h_range)]

    ans = np.vstack((ans_x, ans_y))
    """
    
    return ans, c_

In [4]:
def print_projection_plt(points, color, image):
    """ project converted velodyne points into camera image """
    
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    for i in range(points.shape[1]):
        cv2.circle(hsv_image, (np.int32(points[0][i]),np.int32(points[1][i])),2, (int(color[i]),255,255),-1)

    return cv2.cvtColor(hsv_image, cv2.COLOR_HSV2RGB)

In [ ]:
#subscribe all needed topics



R= np.array([[7.533745e-03, -9.999714e-01, -6.166020e-04], 
            [1.480249e-02, 7.280733e-04, -9.998902e-01],
            [9.998621e-01, 7.523790e-03, 1.480755e-02]])
T= np.array([[-4.069766e-03],
            [-7.631618e-02],
            [-2.717806e-01]])

image_pub = rospy.Publisher("projected_image",Image)
def callback(l_image, camera_info, lidar):
        global image_pub
        global P
        try:    
            bridge = CvBridge()
            cv_image = bridge.imgmsg_to_cv2(l_image, "bgr8")
            points = pointclouds.pointcloud2_to_xyz_array(lidar)
        except CvBridgeError as e:
            print(e)
        
        
        
        print points.shape
        
        P = np.array((camera_info.P))
        P = P.reshape((3,4))
        P = P[:3, :3] #erase fourth column
        ans, c_ = velo3d_2_camera2d_points(points, v_fov=(-24.9, 2.0), h_fov=(-45,45), \
                               vc_path=[R,T], cc_path=P, mode='02')
        projected_image = print_projection_plt(points=ans, color=c_, image=cv_image)
        image_pub.publish(bridge.cv2_to_imgmsg(projected_image, "bgr8"))
        print (len(c_),len(ans))


def listener():
        rospy.init_node('listener')
        print "loop1"
        l_image = message_filters.Subscriber("/kitti/camera_color_left/image_raw", Image )
        camera_info = message_filters.Subscriber("/kitti/camera_gray_left/camera_info", CameraInfo)
        lidar = message_filters.Subscriber("/kitti/velo/pointcloud", PointCloud2)
        ts = message_filters.ApproximateTimeSynchronizer([l_image, camera_info, lidar], 10, 0.1)
        ts.registerCallback(callback)
#         print "loop1"
        rospy.spin()

if __name__ == '__main__':
        listener()


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:12: SyntaxWarning: The publisher should be created with an explicit keyword argument 'queue_size'. Please see http://wiki.ros.org/rospy/Overview/Publishers%20and%20Subscribers for more information.
  if sys.path[0] == '':


loop1
(123397, 3)


/usr/lib/python2.7/dist-packages/numpy/core/numeric.py:301: FutureWarning: in the future, full((1, 32005), 1) will return an array of dtype('int64')
  format(shape, fill_value, array(fill_value).dtype), FutureWarning)


(32005, 2)
(123683, 3)
(32050, 2)
(123783, 3)


/usr/lib/python2.7/dist-packages/numpy/core/numeric.py:301: FutureWarning: in the future, full((1, 32050), 1) will return an array of dtype('int64')
  format(shape, fill_value, array(fill_value).dtype), FutureWarning)
/usr/lib/python2.7/dist-packages/numpy/core/numeric.py:301: FutureWarning: in the future, full((1, 32088), 1) will return an array of dtype('int64')
  format(shape, fill_value, array(fill_value).dtype), FutureWarning)


...

(31883, 2)
(124122, 3)
(31884, 2)
(124048, 3)
(31843, 2)
(123902, 3)
(31757, 2)
(123801, 3)
(31730, 2)
(123704, 3)
(31500, 2)
(123871, 3)
(31433, 2)
(123838, 3)
(31379, 2)
(123925, 3)
(31245, 2)
(123983, 3)
(31254, 2)
(123526, 3)
(31108, 2)
(123460, 3)
(30971, 2)
(123467, 3)


/usr/lib/python2.7/dist-packages/numpy/core/numeric.py:301: FutureWarning: in the future, full((1, 30971), 1) will return an array of dtype('int64')
  format(shape, fill_value, array(fill_value).dtype), FutureWarning)


(30895, 2)
(123208, 3)
(30713, 2)
(123008, 3)


/usr/lib/python2.7/dist-packages/numpy/core/numeric.py:301: FutureWarning: in the future, full((1, 30713), 1) will return an array of dtype('int64')
  format(shape, fill_value, array(fill_value).dtype), FutureWarning)


(30483, 2)
(122898, 3)
(30483, 2)
(122624, 3)
(30391, 2)
(122319, 3)
(30249, 2)
(122125, 3)
(30116, 2)
(121519, 3)
(30097, 2)
(121178, 3)
(30313, 2)
(121210, 3)
(30442, 2)
(120825, 3)
(31100, 2)
(120989, 3)
(31123, 2)
(121283, 3)
(30959, 2)
(121649, 3)
(30623, 2)
(121691, 3)
(30485, 2)
(121482, 3)
(30643, 2)
(120873, 3)
(30807, 2)
(120271, 3)
(30959, 2)
(119588, 3)
(31242, 2)
(118572, 3)
(31214, 2)
(120049, 3)
(30523, 2)
(118371, 3)
(31216, 2)
(120163, 3)
(30400, 2)
(118451, 3)
(31202, 2)
(120212, 3)
(30221, 2)
(118494, 3)
(31109, 2)
(119597, 3)
(29365, 2)
(118765, 3)
(31016, 2)
(118846, 3)
(28562, 2)
(118930, 3)
(30986, 2)
(118423, 3)
(28416, 2)
(119092, 3)
(30902, 2)


In [11]:
P.shape

(3, 4)

In [2]:
"""
header: 
  seq: 0
  stamp: 
    secs: 1317022472
    nsecs: 351949930
  frame_id: "camera_gray_left"
height: 1242
width: 375
distortion_model: "plumb_bob"
D: [-0.3728755, 0.2037299, 0.002219027, 0.001383707, -0.07233722]
K: [984.2439, 0.0, 690.0, 0.0, 980.8141, 233.1966, 0.0, 0.0, 1.0]
R: [0.9999239, 0.00983776, -0.007445048, -0.009869795, 0.9999421, -0.004278459, 0.007402527, 0.004351614, 0.9999631]
P: [721.5377, 0.0, 609.5593, 0.0, 0.0, 721.5377, 172.854, 0.0, 0.0, 0.0, 1.0, 0.0]
binning_x: 0
binning_y: 0
roi:
  x_offset: 0
  y_offset: 0
  height: 0
  width: 0
  do_rectify: False
"""